In [1]:
# 1. Add your previous structure optimization and Hessian calculation code as
# functions in `util.py`
import solution_util as util

In [2]:
# 2. Calculate the Hessian of a molecule at its equilibrium structure with
# GFN2-xTB.
from protomol import smiles
import numpy

smi = "O"  # SMILES formula for propyl radical
prog = "crest"
model = {"method": "gfn2"}
hess_path = "./hess.dat"
recalculate = True

# Generate a starting geometry for the SMILES formula
geo0 = smiles.geometry(smi)

# Optimize the equilibrium structure
geo = util.equilibrium_geometry(geo0, prog=prog, model=model)

if recalculate:
    # Calculate the Hessian
    # hess = util.hessian(geo0, prog=prog, model=model)
    hess = util.hessian_direct(geo0, prog=prog, model=model)
    numpy.savetxt(hess_path, hess)

In [3]:
# 3. Use `mendeleev` to get the isotopic masses of the atoms in your molecule.
from mendeleev import element
from protomol import geom


def isotopic_mass(symbol: str) -> float:
    """Get the isotopic mass of an element by symbol."""
    # 1. Determine the most abundant natural isotope
    mass_number = element(symbol).mass_number
    isotope = element(symbol).isotope(mass_number)
    # 2. Return its mass
    return isotope.mass


masses = list(map(isotopic_mass, geom.symbols(geo)))
masses

[15.99491461926, 1.007825031898, 1.007825031898]

In [4]:
# 4. Mass-weight the Hessian matrix.
H = numpy.loadtxt(hess_path)
M = numpy.diag(numpy.repeat(numpy.sqrt(masses) ** -1, 3))
H_ = M @ H @ M

In [5]:
# 5. Diagonalize the Hessian matrix.
eig_vals, eig_vecs = numpy.linalg.eigh(H_)
eig_vals

array([-4.42167247e-08, -9.88605723e-10,  1.36839611e-09,  9.78714498e-03,
        1.23172365e-02,  1.45633012e-02,  9.54053927e-02,  3.88892647e-01,
        4.25569812e-01])

In [6]:
# 6. Calculate the normal modes of vibration in Cartesian coordinates from your
# eigenvectors.
Q = M @ eig_vecs

In [7]:
# 7. Calculate the resonance frequencies of these normal modes in wavenumbers cm^-1.
import pint

k_vals = pint.Quantity(eig_vals, "hartree / bohr^2 / dalton").m_as("J / m^2 / kg")
f_vals = numpy.sqrt(numpy.clip(k_vals, a_min=0, a_max=None)) / (2 * numpy.pi)
w_vals = f_vals / pint.Quantity("speed_of_light").m_as("cm / s")
w_vals.tolist()

[0.0,
 0.0,
 0.19015606951947703,
 508.5483953226125,
 570.5069222419068,
 620.3462965266586,
 1587.7815060162307,
 3205.6724398173415,
 3353.4333873855207]

In [8]:
# 8. Write a function to visualize a normal mode using `py3Dmol`.
from numpy.typing import ArrayLike
import py3Dmol


def xyz_string_with_mode(geo: geom.Geometry, mode: ArrayLike) -> str:
    """Serialize as xyz string including vibrational mode for animation."""
    natms = geom.count(geo)
    symbs = geom.symbols(geo)
    xyzs = geom.coordinates(geo, unit="angstrom")
    mode_xyzs = numpy.reshape(mode, (natms, 3)).tolist()
    return f"{natms}\n\n" + "\n".join(
        f"{s} {x:10.6f} {y:10.6f} {z:10.6f} {mx:10.6f} {my:10.6f} {mz:10.6f}"
        for s, (x, y, z), (mx, my, mz) in zip(symbs, xyzs, mode_xyzs, strict=True)
    )


def animate_mode(geo: geom.Geometry, mode: ArrayLike) -> None:
    """Animate vibrational mode."""
    xyz_str = xyz_string_with_mode(geo, mode=mode)

    view = py3Dmol.view(width=400, height=400)
    view.addModel(xyz_str, "xyz", {"vibrate": {"frames": 10, "amplitude": 1}})
    view.setStyle({"stick": {}, "sphere": {"scale": 0.3}})
    view.animate({"loop": "backAndForth"})
    view.zoomTo()
    view.show()


# Note: The O-H stretching modes are unphysical (presumably an artifact of xTB?)
print(animate_mode(geo, Q[:, 8]))

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

None
